In [4]:
from astropy.coordinates import SkyCoord
import astropy.units as u
import pandas as pd
import numpy as np
import psrqpy
from collections import Counter

In [6]:
#Getting the data
df = pd.read_csv("New_Sources")
ra = pd.to_numeric(df["RA (deg)"], errors="coerce")
dec = pd.to_numeric(df["DEC (deg)"], errors="coerce")

# Keep only valid entries
valid = ~ra.isna() & ~dec.isna()
ra = ra[valid].values
dec = dec[valid].values

#Query ATNF pulsar data
query = psrqpy.QueryATNF(params=["JNAME", "RAJ", "DECJ", "P0", "DM"])
atnf_table = query.table 
atnf = atnf_table.to_pandas()  

atnf
#Convert to SkyCoord
atnf_coords = SkyCoord(atnf["RAJ"].values, atnf["DECJ"].values, unit=(u.hourangle, u.deg))
my_coords = SkyCoord(ra*u.deg, dec*u.deg)

#Step 4: Crossmatch (find nearest pulsar for each coordinate)
idx, sep2d, _ = my_coords.match_to_catalog_sky(atnf_coords)

# Step 5: Apply a separation limit (optional)
max_sep = 10* u.arcsec  # 0.1° = 6 arcmin
matches = sep2d < max_sep

# Step 6: Build result table
results = pd.DataFrame({
    "My_RA": ra,
    "My_DEC": dec,
    "Nearest_Pulsar": atnf["JNAME"].iloc[idx].values,
    "Separation": sep2d.arcsec,
    "P0": atnf["P0"].iloc[idx].values,
    "DM": atnf["DM"].iloc[idx].values
})

matched = results[matches]

print("Crossmatched Pulsars:")
print(matched)

Crossmatched Pulsars:
           My_RA    My_DEC Nearest_Pulsar  Separation        P0       DM
1465  164.494688 -52.44792     J1057-5226    4.809149  0.197116  29.6835
